In [ ]:
import heapq, math
from collections import Counter, defaultdict

In [ ]:
class Nodo:
    def __init__(self, simbolo=None, frecuencia=0):
        self.simbolo = simbolo
        self.frec = frecuencia
        self.izq = None
        self.der = None
    def __lt__(self, other):
        return self.frec < other.frec

In [ ]:
def crear_arbol(frecs):
    heap = [Nodo(sim, f) for sim, f in frecs.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        n1, n2 = heapq.heappop(heap), heapq.heappop(heap)
        union = Nodo(frecuencia=n1.frec+n2.frec)
        union.izq, union.der = n1, n2
        heapq.heappush(heap, union)
    return heap[0]

def generar_codigos(nodo, prefijo="", codigos={}):
    if nodo.simbolo is not None:
        codigos[nodo.simbolo] = prefijo
    else:
        generar_codigos(nodo.izq, prefijo+"0", codigos)
        generar_codigos(nodo.der, prefijo+"1", codigos)
    return codigos

def independientes(path):
    data = open(path,"rb").read()
    n = len(data)
    frecuencias = Counter(data)

    # Probabilidades
    probabilidades = {simbolo: frecuencia/n for simbolo,frecuencia in frecuencias.items()}

    # Entropía
    H = -sum(p*math.log2(p) for p in probabilidades.values())

    # Huffman
    arbol = crear_arbol(frecuencias)
    codes = generar_codigos(arbol)

    # Longitud promedio
    L = sum(probabilidades[s]*len(codes[s]) for s in codes)

    # Eficiencia y redundancia
    eta = H/L
    R = 1 - eta

    print(f"Entropía H = {H:.4f} bits/símbolo")
    print(f"Longitud promedio L = {L:.4f} bits/símbolo")
    print(f"Eficiencia η = {eta*100:.2f}%")
    print(f"Redundancia R = {R*100:.2f}%")

# Ejemplo
independientes("test.txt")


Entropía H = 1.2988 bits/símbolo
Longitud promedio L = 1.3750 bits/símbolo
Eficiencia η = 94.46%
Redundancia R = 5.54%


In [ ]:
def generate_codes(node, prefix=""):
    """Devuelve un nuevo dict con los códigos (no usa mutable default)."""
    codebook = {}
    # caso especial: árbol con un solo nodo (leaf sin hijos)
    if node.izq is None and node.der is None:
        # si prefix == "" -> asignamos "0" para que el código tenga longitud 1
        codebook[node.simbolo] = prefix if prefix != "" else "0"
        return codebook

    def _rec(n, pre):
        if n.simbolo is not None:
            codebook[n.simbolo] = pre
            return
        if n.izq is not None:
            _rec(n.izq, pre + "0")
        if n.der is not None:
            _rec(n.der, pre + "1")

    _rec(node, prefix)
    return codebook

def dependientes(path):
    data = open(path, "rb").read()
    n = len(data)
    if n < 2:
        print("Archivo muy corto para Markov orden 1.")
        return

    singles = Counter(data)
    bigrams = Counter((data[i], data[i+1]) for i in range(n-1))

    # prob condicionales por estado
    cond = defaultdict(dict)
    for (a,b), f in bigrams.items():
        cond[a][b] = f / singles[a]

    # construir códigos por estado usando *frecuencias* (mejor usar counts para Huffman)
    codes = {}
    for a, following in cond.items():
        # convertir probabilidades a "pesos" proporcionales (o usar counts)
        # aquí usamos los recuentos reales de bigramas:
        frecs = {b: bigrams[(a,b)] for b in following.keys()}
        tree = crear_arbol(frecs)
        codes[a] = generate_codes(tree)

    # Longitud promedio LM (ponderada por prob conjunta P(a,b) = bigrams/(n-1))
    LM = 0.0
    for (a,b), cnt in bigrams.items():
        prob_ab = cnt / (n-1)
        LM += prob_ab * len(codes[a][b])

    # Entropía condicional H(SM)
    HSM = 0.0
    for (a,b), cnt in bigrams.items():
        prob_ab = cnt / (n-1)
        p_cond = cnt / singles[a]   # P(b|a)
        # cuidado si p_cond == 0 (no debería)
        HSM -= prob_ab * math.log2(p_cond)

    eta = HSM / LM if LM > 0 else float('inf')
    R = 1 - eta

    print(f"Entropía condicional H(SM) = {HSM:.4f} bits/símbolo")
    print(f"Longitud promedio LM = {LM:.4f} bits/símbolo")
    print(f"Eficiencia η = {eta*100:.2f}%")
    print(f"Redundancia R = {R*100:.2f}%")

# uso
dependientes("test.txt")


Entropía condicional H(SM) = 0.7058 bits/símbolo
Longitud promedio LM = 1.0000 bits/símbolo
Eficiencia η = 70.58%
Redundancia R = 29.42%
